SHAKA ARISYA - A11.2021.13406
Data Encryption Standard (DES)

Inputan Plaintext dan Kunci(Key)

In [456]:
# Prompt user to enter plaintext and key
plainText = input("Masukkan plaintext (8 karakter atau 64 bits atau 8 bytes) : ")
key = input("Masukkan kunci (16 karakter atau 64 bits atau 8 bytes) Dalam Hexadecimal : ")

# Validate user input
if len(plainText) != 8:
  print("Plaintext harus 8 bytes!")
elif len(key.split()) != 8:
  print("Kunci harus terdiri dari 16 karakter heksadesimal (0-9, A-F) dengan spasi. ")
else:
  # Get ASCII value of each character using ord() function
  ascii_value = [ord(char) for char in plainText]
  print("Plaintext = ", plainText)
  print("Key = ", key)


Masukkan plaintext (8 karakter atau 64 bits atau 8 bytes) : COMPUTER
Masukkan kunci (16 karakter atau 64 bits atau 8 bytes) Dalam Hexadecimal : 13 34 57 79 9B BC DF F1
Plaintext =  COMPUTER
Key =  13 34 57 79 9B BC DF F1


Ubah Plaintext menjadi ASCII value

In [457]:
# Dapatkan value ASCII tiap char menggunakan fungsi ord()
ascii_value = []

for i in range(len(plainText)):
    ascii_value.append(ord(plainText[i]))

# Print each character of plaintext and its corresponding ASCII value
for i in range(len(plainText)):
  print(f"{plainText[i]}: {ascii_value[i]}")


C: 67
O: 79
M: 77
P: 80
U: 85
T: 84
E: 69
R: 82


Setelah mendapat hasil value ASCII konversikan ke dalam binary

In [458]:
def dec2bin(num):
  result = bin(num).replace("0b", "")
  if(len(result) % 4 != 0):
    div = len(result) / 4
    div = int(div)
    counter = (4 * (div + 1)) - len(result)
    for i in range(0, counter):
      result = '0' + result
  return result

In [459]:
ascii_binary = []
# Convert each character of plaintext to binary and store in ascii_binary
for i in range(len(plainText)):
  ascii_binary.append(dec2bin(ascii_value[i]).zfill(8))
# Combine binary strings for each character of plaintext
binary_str = ''.join(ascii_binary)
# Print each character of plaintext and its corresponding binary representation
for i in range(len(plainText)):
  print(f"{plainText[i]}: {binary_str[i*8:(i+1)*8]}")


C: 01000011
O: 01001111
M: 01001101
P: 01010000
U: 01010101
T: 01010100
E: 01000101
R: 01010010


In [460]:
ascii_binary = "".join(ascii_binary)


In [461]:
formatted_ascii_binary = ' '.join([ascii_binary[i:i+8] for i in range(0, len(ascii_binary), 8)])
print("P = ",formatted_ascii_binary)


P =  01000011 01001111 01001101 01010000 01010101 01010100 01000101 01010010


**Langkah 1 : Initial Permutation**

Mengatur ulang posisi semua bit teks biasa menjadi matriks initial_perm(8x8)

Inisialisasi Table Permutation yang pertama

In [462]:
initial_perm = [58, 50, 42, 34, 26, 18, 10, 2,
                60, 52, 44, 36, 28, 20, 12, 4,
                62, 54, 46, 38, 30, 22, 14, 6,
                64, 56, 48, 40, 32, 24, 16, 8,
                57, 49, 41, 33, 25, 17, 9, 1,
                59, 51, 43, 35, 27, 19, 11, 3,
                61, 53, 45, 37, 29, 21, 13, 5,
                63, 55, 47, 39, 31, 23, 15, 7]

In [463]:
def initial_permutation(plain_text, initial_perm, no_bits):
  permutation = ""
  for i in range(0, no_bits):
    permutation += plain_text[initial_perm[i] - 1]
  return permutation

Hasil Initial Permutasi

In [464]:
ascii_perm = initial_permutation(ascii_binary, initial_perm, 64)
ascii_perm

'1111111110111000011101100101011100000000000000000000011010000011'

In [465]:
formatted_ascii_binary = ' '.join([ascii_binary[i:i+8] for i in range(0, len(ascii_binary), 8)])
print("P  = ",formatted_ascii_binary)
print("Hasil Akhir Permutasi Awal")
formatted_ascii_binary = ' '.join([ascii_perm[i:i+8] for i in range(0, len(ascii_perm), 8)])
print("IP = ",formatted_ascii_binary)

P  =  01000011 01001111 01001101 01010000 01010101 01010100 01000101 01010010
Hasil Akhir Permutasi Awal
IP =  11111111 10111000 01110110 01010111 00000000 00000000 00000110 10000011


**Langkah 2 : Pecah hasil Initial Permutasi menjadi 2 bagian yang sama panjang (32 bit)**

Bagi ascii_perm menjadi 2 bagian yang sama panjang (32 bit)

In [466]:
half_length_plaintext = len(ascii_perm) // 2
Lo = ascii_perm[:half_length_plaintext]
Ro = ascii_perm[half_length_plaintext:]

In [467]:
print("L0 = ", ' '.join([Lo[i:i+8] for i in range(0, len(Lo), 8)]))
print("R0 = ", ' '.join([Ro[i:i+8] for i in range(0, len(Ro), 8)]))


L0 =  11111111 10111000 01110110 01010111
R0 =  00000000 00000000 00000110 10000011


BREAK - Berhenti untuk proses permutasi plaintext

**Langkah 3 : Pembangkitan Kunci**

Asumsikan key generator berupa bilangan hexadecimal

In [468]:
def hex2bin(hex_string):
  try:
    # decimal -> biner
    binary_num = bin(hex_string)

    # Menghilangkan awalan '0b' dan mengonversi ke bentuk 8 digit
    binary_result = '{:08b}'.format(int(binary_num, 2))

    return binary_result
  except ValueError:
    return "Input bukan bilangan heksadesimal yang valid"

Ubah value hexadecimal -> decimal

In [469]:
key_list = [int(hex_value, 16) for hex_value in key.split()]
key_list

[19, 52, 87, 121, 155, 188, 223, 241]

Ubah value decimal tersebut -> binary

In [470]:
key_binary = []
for i in range(len(key_list)):
  key_binary.append(hex2bin(key_list[i]))
key_binary

['00010011',
 '00110100',
 '01010111',
 '01111001',
 '10011011',
 '10111100',
 '11011111',
 '11110001']

In [471]:
key_binary = "".join(key_binary)

In [472]:
formatted_key_binary = ' '.join([key_binary[i:i+8] for i in range(0, len(key_binary), 8)])
print("K  = ",formatted_key_binary)

K  =  00010011 00110100 01010111 01111001 10011011 10111100 11011111 11110001


**Table PC-1**

In [473]:
matrix_pc_1 = [ 57, 49, 41, 33, 25, 17, 9,
                1, 58, 50, 42, 34, 26, 18,
                10, 2, 59, 51, 43, 35, 27,
                19, 11, 3, 60, 52, 44, 36,
                63, 55, 47, 39, 31, 23, 15,
                7, 62, 54, 46, 38, 30, 22,
                14, 6, 61, 53, 45, 37, 29,
                21, 13, 5, 28, 20, 12, 4 ]

**Langkah 4 : Permutasi PC-1**

In [474]:
key_perm = initial_permutation(key_binary, matrix_pc_1, 56)


In [475]:
formatted_key_binary = ' '.join([key_binary[i:i+8] for i in range(0, len(key_binary), 8)])
print("K   = ",formatted_key_binary)
formatted_key_perm = ' '.join([key_perm[i:i+7] for i in range(0, len(key_perm), 7)])
print("K+  = ",formatted_key_perm)

K   =  00010011 00110100 01010111 01111001 10011011 10111100 11011111 11110001
K+  =  1111000 0110011 0010101 0101111 0101010 1011001 1001111 0001111


**Langkah 5 :  Pecah hasil key_perm menjadi 2 bagian yang sama panjang (28 bit)**

In [476]:
half_length_key = len(key_perm) // 2
Co = key_perm[:half_length_key]
Do = key_perm[half_length_key:]

In [477]:
print("C0 = ", ' '.join([Co[i:i+7] for i in range(0, len(Co), 7)]))
print("D0 = ", ' '.join([Do[i:i+7] for i in range(0, len(Do), 7)]))

C0 =  1111000 0110011 0010101 0101111
D0 =  0101010 1011001 1001111 0001111


**Langkah 6 :** *Wrapping* atau *Round-Shift*

In [478]:
shift_table=[ 1, 1, 2, 2,
              2, 2, 2, 2,
              1, 2, 2, 2,
              2, 2, 2, 1 ]

**Table PC-2**

In [479]:
matrix_pc_2 = [ 14, 17, 11, 24, 1, 5,
                3, 28, 15, 6, 21, 10,
                23, 19, 12, 4, 26, 8,
                16, 7, 27, 20, 13, 2,
                41, 52, 31, 37, 47, 55,
                30, 40, 51, 45, 33, 48,
                44, 49, 39, 56, 34, 53,
                46, 42, 50, 36, 29, 32 ]

**Langkah 7 : Lakukan pergeseran Co dan Do ke kiri sesuai dengan shift table**

In [480]:
def shift_left(kunci,count_shift):
  left_shift = ""
  for i in range(count_shift): # Loop shift table
    for j in range(1, len(kunci)): # Loop index dari key
      left_shift += kunci[j]
    left_shift += kunci[0]
    kunci = left_shift
    left_shift = ""
  return kunci

**Langkah 8 : Hasil perolehan dari Cn dan Dn di permutasikan dengan table permutasi PC-2**

In [481]:
# Key Transformation
cn_shift = Co
dn_shift = Do
# Save key transformation
list_combine_perm = []
decrypt = ""
for i in range(0, 16):
  Cn = shift_left(cn_shift, shift_table[i])
  Dn = shift_left(dn_shift, shift_table[i])
  combine = Cn + Dn

  # Permutasi kan dengan table PC-2
  combine_perm = initial_permutation(combine, matrix_pc_2, 48)
  list_combine_perm.append(combine_perm)
  decrypt = list_combine_perm[::-1]

  print(f"Pergeseran {i+1}")
  print("C" + str(i+1) + "\t=", Cn)
  print("D" + str(i+1) + "\t=", Dn)
  print("C" + str(i+1) + "D" + str(i+1) + "\t=", combine)
  print()

  cn_shift = Cn
  dn_shift = Dn

Pergeseran 1
C1	= 1110000110011001010101011111
D1	= 1010101011001100111100011110
C1D1	= 11100001100110010101010111111010101011001100111100011110

Pergeseran 2
C2	= 1100001100110010101010111111
D2	= 0101010110011001111000111101
C2D2	= 11000011001100101010101111110101010110011001111000111101

Pergeseran 3
C3	= 0000110011001010101011111111
D3	= 0101011001100111100011110101
C3D3	= 00001100110010101010111111110101011001100111100011110101

Pergeseran 4
C4	= 0011001100101010101111111100
D4	= 0101100110011110001111010101
C4D4	= 00110011001010101011111111000101100110011110001111010101

Pergeseran 5
C5	= 1100110010101010111111110000
D5	= 0110011001111000111101010101
C5D5	= 11001100101010101111111100000110011001111000111101010101

Pergeseran 6
C6	= 0011001010101011111111000011
D6	= 1001100111100011110101010101
C6D6	= 00110010101010111111110000111001100111100011110101010101

Pergeseran 7
C7	= 1100101010101111111100001100
D7	= 0110011110001111010101010110
C7D7	= 110010101010111111110000110001100111

Key hasil permutasi dengan table PC-2

In [482]:

for i in range(len(list_combine_perm)):
    formatted_key_perm = ' '.join([list_combine_perm[i][j:j+6] for j in range(0, len(list_combine_perm[i]), 6)])
    print("K" + str(i + 1) + "\t=", formatted_key_perm)


K1	= 000110 110000 001011 101111 111111 000111 000001 110010
K2	= 011110 011010 111011 011001 110110 111100 100111 100101
K3	= 010101 011111 110010 001010 010000 101100 111110 011001
K4	= 011100 101010 110111 010110 110110 110011 010100 011101
K5	= 011111 001110 110000 000111 111010 110101 001110 101000
K6	= 011000 111010 010100 111110 010100 000111 101100 101111
K7	= 111011 001000 010010 110111 111101 100001 100010 111100
K8	= 111101 111000 101000 111010 110000 010011 101111 111011
K9	= 111000 001101 101111 101011 111011 011110 011110 000001
K10	= 101100 011111 001101 000111 101110 100100 011001 001111
K11	= 001000 010101 111111 010011 110111 101101 001110 000110
K12	= 011101 010111 000111 110101 100101 000110 011111 101001
K13	= 100101 111100 010111 010001 111110 101011 101001 000001
K14	= 010111 110100 001110 110111 111100 101110 011100 111010
K15	= 101111 111001 000110 001101 001111 010011 111100 001010
K16	= 110010 110011 110110 001011 000011 100001 011111 110101


Hasil permutasi plainteks sebelumnya

In [483]:
print("L0 = ", ' '.join([Lo[i:i+8] for i in range(0, len(Lo), 8)]))
print("R0 = ", ' '.join([Ro[i:i+8] for i in range(0, len(Ro), 8)]))

L0 =  11111111 10111000 01110110 01010111
R0 =  00000000 00000000 00000110 10000011


**Proses Enchipering**

In [484]:
# XOR function
def xor(a,b):
  # a and b should be in binary
  ans = ""
  for i in range(len(a)):
    if(a[i] == b[i]): #Li = Ri-1
      ans += "0"
    else:
      ans += "1"
  return ans

Function Konversi Binary -> Decimal

In [485]:
def bin2dec(msg):
  decimal, i = 0,0
  while(msg != 0):
    dec = msg % 10
    decimal += dec * pow(2, i)
    msg = msg // 10
    i += 1
  return decimal

**Inisialisasi Table Ekspansi**

In [486]:
# Table Ekspansi Ro (48 bit)
expand_perm = [ 32, 1, 2, 3, 4, 5,
                4, 5, 6, 7, 8, 9,
                8, 9, 10, 11, 12, 13,
                12, 13, 14, 15, 16, 17,
                16, 17, 18, 19, 20, 21,
                20, 21, 22, 23, 24, 25,
                24, 25, 26, 27, 28, 29,
                28, 29, 30, 31, 32, 1 ]

**Inisialisasi S-Box**

In [487]:
s_box = [
          # S-Box 1
        [[14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
         [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
         [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
         [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]],
          # S-Box 2
        [[15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
         [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
         [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
         [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]],
          # S-Box 3
        [[10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
         [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
         [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
         [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12]],
          # S-Box 4
        [[7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
         [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
         [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
         [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14]],
          # S-Box 5
        [[2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
         [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
         [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
         [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3]],
          # S-Box 6
        [[12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
         [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
         [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
         [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13]],
          # S-Box 7
        [[4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
         [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
         [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
         [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12]],
          # S-Box 8
        [[13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
         [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
         [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
         [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11]]
      ]

In [488]:
# Permutation table
perm_table=[16, 7, 20, 21, 29, 12, 28, 17,
            1, 15, 23, 26, 5, 18, 31, 10,
            2, 8, 24, 14, 32, 27, 3, 9,
            19, 13, 30, 6, 22, 11, 4, 25]

1.  Expansi R0 dengan table Expansi

expansi pada blok Ri-1 32 bit menjadi blok 48 bit

In [489]:
right = Ro  # Assuming Ro is some initial value
# Start from 1 to modify ER2 to ER16
right_expand = initial_permutation(right, expand_perm, 48)  # Assuming you have a function for initial_permutation
formatted_right_expand = ' '.join([right_expand[j:j+6] for j in range(0, len(right_expand), 6)])
print("Hasil expansi R0 terhadap Table Expansi ")
print("E(R0)"  + "\t=", formatted_right_expand)


Hasil expansi R0 terhadap Table Expansi 
E(R0)	= 100000 000000 000000 000000 000000 001101 010000 000110


2. Fungsi XOR dari ekspansi R0 dan Key 1 untuk menghasilkan A1

In [490]:
right_expand = initial_permutation(right, expand_perm, 48)
A = xor(right_expand, list_combine_perm[0])
formatted_A = ' '.join([A[i:i+6] for i in range(0, len(A), 6)])
print("E(R0)"  + "\t=", formatted_right_expand)
formatted_key_perm = ' '.join([list_combine_perm[0][j:j+6] for j in range(0, len(list_combine_perm[0]), 6)])
print("K1" + "\t=", formatted_key_perm)
print("____________________________________________________________________XOR")
print("A1" + "\t=", formatted_A)

E(R0)	= 100000 000000 000000 000000 000000 001101 010000 000110
K1	= 000110 110000 001011 101111 111111 000111 000001 110010
____________________________________________________________________XOR
A1	= 100110 110000 001011 101111 111111 001010 010001 110100


3. Proses substitusi nilai dengan table S-Box (1)

In [491]:
print("Proses Substitusi S-BOX(1)")
print("A1" + "\t=", formatted_A)
print("Bytes A1 Yang di substitusi dengan table S-BOX(1)")
first_byte_A = A[:6]
formatted_first_byte_A = ' '.join([first_byte_A[i:i+6] for i in range(0, len(first_byte_A), 6)])
print("A1(1)"  + "\t=", formatted_first_byte_A )
s_box_str = ""
list_s_box = []
for j in range(0, 8):
    row = bin2dec(int(A[j * 6] + A[j * 6 + 5]))
    column = bin2dec(int(A[j * 6 + 1] + A[j * 6 + 2] + A[j * 6 + 3] + A[j * 6 + 4]))
    value = s_box[j][row][column]
    s_box_str += dec2bin(value)
list_s_box.append(s_box_str)
formatted_s_box = ' '.join([list_s_box[0][j:j+4] for j in range(0, len(list_s_box[0]), 4)])
print("---------------------------------------------------------------------")
print("Hasil S-box substitution B1")
print("B1" + "\t=", formatted_s_box)

Proses Substitusi S-BOX(1)
A1	= 100110 110000 001011 101111 111111 001010 010001 110100
Bytes A1 Yang di substitusi dengan table S-BOX(1)
A1(1)	= 100110
---------------------------------------------------------------------
Hasil S-box substitution B1
B1	= 1000 0101 0100 1000 0011 0010 1110 1010


4. Permutasikan hasil dari substitusi table S-Box Menjadi Hasil P-BOX (1)

In [492]:
p_box = []
left = Lo
left_t = []
right_t = []
p_box.append(initial_permutation(list_s_box[0], perm_table, 32))
p_transform = p_box[0]
left_s = right # L1 = R0
left_t.append(left_s)
right = xor(left, p_transform) # R1)
right_t.append(right)
formatted_p_box = ' '.join([p_box[0][j:j+8] for j in range(0, len(p_box[0]), 8)])
print("B1" + "\t=", formatted_s_box)
print("P(B1)"  + "\t=", formatted_p_box)

B1	= 1000 0101 0100 1000 0011 0010 1110 1010
P(B1)	= 00101000 10110011 01000100 11010001


5. Mendapatkan nilai Left (L) dan Right (R) sebanyak 16 kali putaran

In [493]:
left = Lo
right = Ro
left_t = []
right_t = []
left_s = right # L1 = R0
left_t.append(left_s)
right = xor(left, p_transform) # R1)
right_t.append(right)
left = right_t[0]
right = xor(left_t[0], p_transform)
left_t.append(right)
right_t.append(left)
left_t.append(left)
right_t.append(right)
print("P(B1)"  + "\t=", formatted_p_box)
formatted_left_t = ' '.join([left_t[0][j:j+8] for j in range(0, len(left_t[0]), 8)])
print("L1" +  "\t=", formatted_left_t)
formatted_right_t = ' '.join([right_t[0][j:j+8] for j in range(0, len(right_t[0]), 8)])
print("____________________________________________________________________XOR")
print("R1" +  "\t=", formatted_right_t )


P(B1)	= 00101000 10110011 01000100 11010001
L1	= 00000000 00000000 00000110 10000011
____________________________________________________________________XOR
R1	= 11010111 00001011 00110010 10000110


**Lakukan Proses Enchipering dengan iterasi sampai dengan putaran ke -16**

In [494]:
s_box_str = ""
list_s_box = []
p_box = []
left = Lo
right = Ro
left_t = []
right_t = []

for i in range(0, 16):
  print("||====Enchipering " + str(i+1)+"====||")
  right_expand = initial_permutation(right, expand_perm, 48)
  A = xor(right_expand, list_combine_perm[i])
  right_expand = initial_permutation(right, expand_perm, 48)
  # Separate output of ER with a space every 6 bits
  formatted_right_expand = ' '.join([right_expand[i:i+6] for i in range(0, len(right_expand), 6)])
  print("ER" + str(i+1) + "\t=", formatted_right_expand)
  # Continue with the rest of the code
  formatted_A = ' '.join([A[i:i+6] for i in range(0, len(A), 6)])
  print("A" + str(i+1) + "\t=", formatted_A)
  # Menghitung row dan column dari s-box
  for j in range(0, 8):
    row = bin2dec(int(A[j * 6] + A[j * 6 + 5]))
    column = bin2dec(int(A[j * 6 + 1] + A[j * 6 + 2] + A[j * 6 + 3] + A[j * 6 + 4]))
    value = s_box[j][row][column]
    s_box_str += dec2bin(value)
  list_s_box.append(s_box_str)
  p_box.append(initial_permutation(list_s_box[i], perm_table, 32))#Ri = Li-1 X pB(i)

  p_transform = p_box[i]
  if i == 0:
    left_s = right # L1 = R0
    left_t.append(left_s)
    right = xor(left, p_transform) # R1)
    right_t.append(right)
  else:
    left = right_t[i-1]
    right = xor(left_t[i-1], p_transform)

    if i == 15:
      left_t.append(right)
      right_t.append(left)
    else:
      left_t.append(left)
      right_t.append(right)
  s_box_str = ""

  formatted_s_box = ' '.join([list_s_box[i][j:j+4] for j in range(0, len(list_s_box[i]), 4)])
  print("B" + str(i+1) + "\t=", formatted_s_box)
  formatted_p_box = ' '.join([p_box[i][j:j+8] for j in range(0, len(p_box[i]), 8)])
  print("PB" + str(i+1) + "\t=", formatted_p_box)
  formatted_left_t = ' '.join([left_t[i][j:j+8] for j in range(0, len(left_t[i]), 8)])
  print("L" + str(i+1) + "\t=", formatted_left_t)
  formatted_right_t = ' '.join([right_t[i][j:j+8] for j in range(0, len(right_t[i]), 8)])
  print("R" + str(i+1) + "\t=", formatted_right_t )
  print()

||====Enchipering 1====||
ER1	= 100000 000000 000000 000000 000000 001101 010000 000110
A1	= 100110 110000 001011 101111 111111 001010 010001 110100
B1	= 1000 0101 0100 1000 0011 0010 1110 1010
PB1	= 00101000 10110011 01000100 11010001
L1	= 00000000 00000000 00000110 10000011
R1	= 11010111 00001011 00110010 10000110

||====Enchipering 2====||
ER2	= 011010 101110 100001 010110 100110 100101 010000 001101
A2	= 000100 110100 011010 001111 010000 011001 110111 101000
B2	= 1101 1100 0100 0011 1000 0000 1111 1001
PB2	= 10001011 11011001 10001100 00010011
L2	= 11010111 00001011 00110010 10000110
R2	= 10001011 11011001 10001010 10010000

||====Enchipering 3====||
ER3	= 010001 010111 111011 110011 110001 010101 010010 100001
A3	= 000100 001000 001001 111001 100001 111001 101100 111000
B3	= 1101 0110 0011 1100 1011 0110 0111 1111
PB3	= 01101111 10110010 10011100 11111110
L3	= 10001011 11011001 10001010 10010000
R3	= 10111000 10111001 10101110 01111000

||====Enchipering 4====||
ER4	= 010111 1100

**Permutasikan value Left (L) dan Right(R) dengan table Invers Permutasi serta proses mendapatkan cipher**

In [495]:
# Final Permutaion Table
final_perm = [40, 8, 48, 16, 56, 24, 64, 32,
              39, 7, 47, 15, 55, 23, 63, 31,
              38, 6, 46, 14, 54, 22, 62, 30,
              37, 5, 45, 13, 53, 21, 61, 29,
              36, 4, 44, 12, 52, 20, 60, 28,
              35, 3, 43, 11, 51, 19, 59, 27,
              34, 2, 42, 10, 50, 18, 58, 26,
              33, 1, 41, 9, 49, 17, 57, 25 ]

In [496]:
plaintext_combine = left_t[15] + right_t[15]
plaintext_combine = left_t[15] + right_t[15]
formatted_plaintext_combine = ' '.join([plaintext_combine[i:i+8] for i in range(0, len(plaintext_combine), 8)])
print("L16 R16 = ", formatted_plaintext_combine)
cipher = ""
plaintext_invers = left_t[15] + right_t[15]
for i in range(len(plaintext_invers)):
    cipher = initial_permutation(plaintext_invers, final_perm, 64)
cipher
formatted_plaintext_invers = ' '.join([cipher[i:i+8] for i in range(0, len(cipher), 8)])
print("Cipher  = ", formatted_plaintext_invers)

L16 R16 =  00011111 10010111 10100101 11100110 01101110 10100010 10101000 10110001
Cipher  =  01010110 11110001 11010101 11001000 01010010 10101111 10000001 00111111


**Decrypt Text**

In [497]:
print(decrypt)

['110010110011110110001011000011100001011111110101', '101111111001000110001101001111010011111100001010', '010111110100001110110111111100101110011100111010', '100101111100010111010001111110101011101001000001', '011101010111000111110101100101000110011111101001', '001000010101111111010011110111101101001110000110', '101100011111001101000111101110100100011001001111', '111000001101101111101011111011011110011110000001', '111101111000101000111010110000010011101111111011', '111011001000010010110111111101100001100010111100', '011000111010010100111110010100000111101100101111', '011111001110110000000111111010110101001110101000', '011100101010110111010110110110110011010100011101', '010101011111110010001010010000101100111110011001', '011110011010111011011001110110111100100111100101', '000110110000001011101111111111000111000001110010']


In [498]:
for i in range(len(decrypt)):
  print("K" + str(i + 1) + "\t=", decrypt[i])

K1	= 110010110011110110001011000011100001011111110101
K2	= 101111111001000110001101001111010011111100001010
K3	= 010111110100001110110111111100101110011100111010
K4	= 100101111100010111010001111110101011101001000001
K5	= 011101010111000111110101100101000110011111101001
K6	= 001000010101111111010011110111101101001110000110
K7	= 101100011111001101000111101110100100011001001111
K8	= 111000001101101111101011111011011110011110000001
K9	= 111101111000101000111010110000010011101111111011
K10	= 111011001000010010110111111101100001100010111100
K11	= 011000111010010100111110010100000111101100101111
K12	= 011111001110110000000111111010110101001110101000
K13	= 011100101010110111010110110110110011010100011101
K14	= 010101011111110010001010010000101100111110011001
K15	= 011110011010111011011001110110111100100111100101
K16	= 000110110000001011101111111111000111000001110010


**Proses Dechipering**

In [499]:
s_box_str = ""
list_s_box = []
p_box = []
left = left_t[15]
right = right_t[15]
left_t = []
right_t = []

for i in range(0, 16):
  right_expand = initial_permutation(right, expand_perm, 48)
  A = xor(right_expand, decrypt[i])

  # Menghitung row dan column dari s-box
  for j in range(0, 8):
    row = bin2dec(int(A[j * 6] + A[j * 6 + 5]))
    column = bin2dec(int(A[j * 6 + 1] + A[j * 6 + 2] + A[j * 6 + 3] + A[j * 6 + 4]))
    value = s_box[j][row][column]
    s_box_str += dec2bin(value)
  list_s_box.append(s_box_str)
  p_box.append(initial_permutation(list_s_box[i], perm_table, 32))
  p_transform = p_box[i]
  if i == 0:
    left_s = right # L1 = R0
    left_t.append(left_s)

    right = xor(left, p_transform) # R1)
    right_t.append(right)
  else:
    left = right_t[i-1]
    right = xor(left_t[i-1], p_transform)

    if i == 15:
      left_t.append(right)
      right_t.append(left)
    else:
      left_t.append(left)
      right_t.append(right)
  s_box_str = ""
  print("L" + str(i+1) + "\t=", left_t[i])
  print("R" + str(i+1) + "\t=", right_t[i])
  print()

L1	= 01101110101000101010100010110001
R1	= 00110110011000001100110100101010

L2	= 00110110011000001100110100101010
R2	= 11001011100001000100010001011101

L3	= 11001011100001000100010001011101
R3	= 00110011101111011111011101100101

L4	= 00110011101111011111011101100101
R4	= 01001101010011111000101110010110

L5	= 01001101010011111000101110010110
R5	= 10011011110101010111100110001100

L6	= 10011011110101010111100110001100
R6	= 00111100011100010011101111000101

L7	= 00111100011100010011101111000101
R7	= 10010111110000000001011110101000

L8	= 10010111110000000001011110101000
R8	= 11100101010010101001100001010001

L9	= 11100101010010101001100001010001
R9	= 10010000111110010011111011001001

L10	= 10010000111110010011111011001001
R10	= 00101101100010110111011000111101

L11	= 00101101100010110111011000111101
R11	= 10110100111000101100110100110001

L12	= 10110100111000101100110100110001
R12	= 10111000101110011010111001111000

L13	= 10111000101110011010111001111000
R13	= 1000101111011001100010101

In [500]:
decipher = ""
plaintext_invers = left_t[15] + right_t[15]
for i in range(len(plaintext_invers)):
  decipher = initial_permutation(plaintext_invers, final_perm, 64)
decipher

'0100001101001111010011010101000001010101010101000100010101010010'

Konversi bilangan Biner -> Teks

In [501]:
partitions_bits = 8 # Menyusun bilangan biner dalam potongan-potongan 8 bit
binary_bits = [decipher[i:i + partitions_bits] for i in range(0, len(decipher), partitions_bits)]
decimal_characters = [int(partitions, 2) for partitions in binary_bits]

# Mengonversi bilangan bulat -> teks
text = ''.join(chr(decimal) for decimal in decimal_characters)

print("Hasil decrypt teks : ", text)

Hasil decrypt teks :  COMPUTER
